In [51]:
# import relevant packages
import torch
import numpy as np
import time

## 1. Multiple views of a storage

In [15]:
data = torch.full((13, 13), 1)
idx_2 = [1, 6, 11]
idx_3 = [3, 4, 8, 9]

data[idx_2, :] = 2
data[:, idx_2] = 2

data[3:5, idx_3] = 3
data[8:10, idx_3] = 3
data[idx_3, 3:5] = 3
data[idx_3, 8:10] = 3

In [16]:
print(data)

tensor([[1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1]])


## 2. Eigendecomposition

In [46]:
M = torch.empty((20, 20)).normal_()
diag = torch.diag(torch.arange(1, 21)).float()
M_inv = torch.inverse(M)

In [47]:
target_matrix = torch.mm(M_inv, torch.mm(diag, M))
e, v = torch.eig(target_matrix)

## 3. Flops per second

In [53]:
M1 = torch.empty((5000, 5000)).normal_()
M2 = torch.empty((5000, 5000)).normal_()

In [56]:
start = time.perf_counter()
torch.mm(M1, M2)
end = time.perf_counter()

# Time complexity for matrix multiplication = O(n^3)
print("Flops per second: {}".format(round(5000**3*(end - start))))

Flops per second: 85791695125


## 4. Playing with strides

In [74]:
def mul_row(m):
    start = time.perf_counter()
    row = m.size(dim=0)
    for idx in range(row):
        temp = m.narrow(0, idx, 1)
        temp *= (idx + 1)
    end = time.perf_counter()
    return (end - start)

In [96]:
def mul_row_fast(m):
    start = time.perf_counter()
    row = m.size(dim=1)
    col = m.size(dim=0)
    
    m_out = torch.mul(torch.arange(1, row+1).expand(col, row).t(), m[0][0])
    end = time.perf_counter()
    return (end - start)

In [97]:
m = torch.full((1000, 400), 2.0)

In [98]:
print("Time used by mul_row: {}".format(mul_row(m)))
print("Time used by mul_row_fast: {}".format(mul_row_fast(m)))

Time used by mul_row: 0.006425214000046253
Time used by mul_row_fast: 0.0005277789969113655
